# 1.2 - Sistemas de recomendación


#### Filtro Colaborativo

Técnica utilizada por algunos sistemas recomendadores, los cuales  suelen incluir conjuntos de datos muy grandes. Hay dos tipos de filtros colaborativos:

+ Basados en Usuarios (User-based):

Se basan en una premisa simple de similitud entre gustos, es decir, en que si una persona A tiene la misma opinión que una persona B sobre un tema, A es más probable que tenga la misma opinión que B en otro tema diferente que la opinión que tendría una persona elegida azar. 

Tiene ciertos inconvenientes. 
Requiere evaluaciones previas, sin algunas evaluaciones de usuarios, no se tienen pistas sobre las evaluaciones de los nuevos usuarios.
No es una matriz densa, usualmente los usuarios dan su rating sobre algunos items y no sobre toda la base de datos, lo que puede dificultar algunos casos por falta de datos pero puede ayudar ya que no se tiene que hacer los cálculos para todas las combinaciones posibles.
Altos costos si los perfiles de usuarios cambian, si los gustos cambian o hay ruido en los datos (ejemplo, 2 personas usan la misma cuenta en el sistema) todo el modelo del sistema debe ser recalculado.

+ Basados en Items (Items-based):

Buscan la similitud entre elementos, utilizan distribuciones de calificación por artículo, no por usuario, lo que puede ayudar a sobrellevar algunos de los problemas de los modelos User-based. Con más usuarios que elementos, cada elemento tiende a tener más calificaciones que cada usuario, por lo que la calificación promedio de un elemento generalmente no cambia rápidamente. Esto conduce a distribuciones de calificación más estables en el modelo, por lo que no es necesario reconstruir el modelo con tanta frecuencia. Cuando los usuarios consumen y luego califican un artículo, los elementos similares de ese elemento se seleccionan del modelo de sistema existente y se agregan a las recomendaciones del usuario.

### Ejemplo Peliculas (User-based)

In [ ]:
import pandas as pd
import pylab as plt

%matplotlib inline

In [ ]:
from scipy.spatial.distance import pdist       # calcula distancias entre puntos

from scipy.spatial.distance import squareform  # convierte a una matriz cuadrada

In [ ]:
ratings=pd.read_csv('../data/movie_ratings.csv').set_index('Movie')

ratings.head()

In [ ]:
def plot(m1, m2):
    x=ratings.T[m1]     # ratings de pelis
    y=ratings.T[m2]

    n=list(ratings.columns)     # nombres de usuarios

    plt.figure(figsize=(10, 5))

    plt.scatter(x, y, s=0)      # scatter vacio

    plt.title('Espacio para {} VS. {}'.format(m1, m2), fontsize=14)
    plt.xlabel(m1, fontsize=14)
    plt.ylabel(m2, fontsize=14)

    for i,e in enumerate(n):
        plt.annotate(e, (x[i], y[i]), fontsize=12)   # escribe nombres

    plt.show();

In [ ]:
plot('Aquaman', 'Deadpool 2')

In [ ]:
plot('Venom', 'Deadpool 2')

In [ ]:
plot('Bohemian Rhapsody', 'Incredibles 2')

In [ ]:
distancias=squareform(pdist(ratings.T, 'euclidean'))    # distancia entre usuarios

distancias   # pertenecen al intervalo [0, +inf)

In [ ]:
similitud = 1 / (1+distancias)

similitud

In [ ]:
similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

similitudes  # pertenecen al intervalo (0, 1)

##### metricas de distancia

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html

https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.squareform.html

https://docs.scipy.org/doc/scipy-0.14.0/reference/spatial.distance.html

**euclidea**

$$d=\sqrt{x^2 + y^2}$$

$$d=\sqrt{x_1^2 + x_2^2 + \ldots + x_n^2}$$

**cityblock - manhattan**

$$d=x + y$$

**similitud del coseno**

**producto escalar**

$$(1, 2, 3)·(2, 1, 1)= 1·2 + 2·1 + 3·1 =7$$

$$\cos{\theta}=\frac{v1·v2}{|v1|·|v2|}$$

donde:

$|V1|=\sqrt{x_1^2+x_2^2+x_3^2}$

### Generar recomendaciones

In [ ]:
tom={'Aquaman': 2, 
     'Avengers: Infinity War':1, 
     'Black Panther':5,
     'Bohemian Rhapsody':5, 
     'Deadpool 2':2,
     'Fantastic Beasts: The Crimes of Grindelwald':3, 
     'Incredibles 2':3,
     'Jurassic World: Fallen Kingdom':4, 
     'Mission: Impossible – Fallout':3,
     'Venom':3}

In [ ]:
ratings['Tom']=pd.Series(tom)

ratings

In [ ]:
distancias=squareform(pdist(ratings.T, 'euclidean')) 

similitud=1/(1+distancias)

similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

similitudes

In [ ]:
n_ratings=pd.read_csv('../data/movie_ratings2.csv').set_index('Movie')  # pelis que Tom NO ha visto

n_ratings

#### Score de similitud

In [ ]:
simil_score=similitudes['Tom'].sort_values(ascending=False)[1:]

simil_score

In [ ]:
reco=n_ratings.copy()

reco  # Tom No ha visto

In [ ]:
for n,s in dict(simil_score).items():
    reco[n]=reco[n]*s   # producto entre similitud y rating
    
reco['Total']=reco.sum(axis=1)

reco.sort_values('Total', ascending=False)

In [ ]:
simil_score

In [ ]:
n_ratings

In [ ]:
# con otra metrica

distancias=squareform(pdist(ratings.T, 'cosine')) 

similitud=1/(1+distancias)

similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

simil_score=similitudes['Tom'].sort_values(ascending=False)[1:]

reco=n_ratings.copy()

for n,s in dict(simil_score).items():
    reco[n]=reco[n]*s
    
reco['Total']=reco.sum(axis=1)
reco.sort_values('Total', ascending=False)

In [ ]:
distancias=squareform(pdist(ratings.T, 'cityblock')) 

similitud=1/(1+distancias)

similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

simil_score=similitudes['Tom'].sort_values(ascending=False)[1:]

reco=n_ratings.copy()

for n,s in dict(simil_score).items():
    reco[n]=reco[n]*s
    
reco['Total']=reco.sum(axis=1)
reco.sort_values('Total', ascending=False)

In [ ]:
distancias=squareform(pdist(ratings.T, 'canberra')) 

similitud=1/(1+distancias)

similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

simil_score=similitudes['Tom'].sort_values(ascending=False)[1:]

reco=n_ratings.copy()

for n,s in dict(simil_score).items():
    reco[n]=reco[n]*s
    
reco['Total']=reco.sum(axis=1)
reco.sort_values('Total', ascending=False)

In [ ]:
distancias=squareform(pdist(ratings.T, 'jaccard')) 

similitud=1/(1+distancias)

similitudes=pd.DataFrame(similitud, index=ratings.columns, columns=ratings.columns)

simil_score=similitudes['Tom'].sort_values(ascending=False)[1:]

reco=n_ratings.copy()

for n,s in dict(simil_score).items():
    reco[n]=reco[n]*s
    
reco['Total']=reco.sum(axis=1)
reco.sort_values('Total', ascending=False)